In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

c:\ProgramData\miniconda3\envs\clip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\clip\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:

add_generation_prompt 参数作用
"""

In [2]:
model_path = f"E:\pythonCode\Qwen\model"

In [3]:
# 导入数据集
# 将json文件转换为DataFrame
df = pd.read_json('huanhuan.json') 
ds = Dataset.from_pandas(df)
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
messages = [
    {"role": "system", "content": "===模拟系统指令==="},
    {"role": "user", "content": "===用户消息测试==="},
    {"role": "assistant", "content": "===模型回复消息测试==="},
]
# 这里是用来测试该模型的语言模版是什么样的，因为要将训练数据格式化为该模型的格式
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
print(text)
"""
输出可以分为四部分，每部分用<|im_start|>开始，以<|im_end|>结尾
第一部分，系统指令
<|im_start|>system
===模拟系统指令===<|im_end|>

第二部分，用户消息
<|im_start|>user
===用户消息测试===<|im_end|>

第三部分，模型回复
<|im_start|>assistant
<think>
这里是因为开启了思考模式 enable_thinking=True
</think>

===模型回复消息测试===<|im_end|>

第四部分，模型回复的生成提示符，add_generation_prompt=True, 告诉模型下一次从这里开始回复，避免重复上面内容
<|im_start|>assistant
"""

<|im_start|>system
===模拟系统指令===<|im_end|>
<|im_start|>user
===用户消息测试===<|im_end|>
<|im_start|>assistant
<think>

</think>

===模型回复消息测试===<|im_end|>
<|im_start|>assistant



'\n输出可以分为四部分，每部分用<|im_start|>开始，以<|im_end|>结尾\n第一部分，系统指令\n<|im_start|>system\n===模拟系统指令===<|im_end|>\n\n第二部分，用户消息\n<|im_start|>user\n===用户消息测试===<|im_end|>\n\n第三部分，模型回复\n<|im_start|>assistant\n<think>\n这里是因为开启了思考模式 enable_thinking=True\n</think>\n\n===模型回复消息测试===<|im_end|>\n\n第四部分，模型回复的生成提示符，告诉模型下一次从这里开始回复，避免重复上面内容\n<|im_start|>assistant\n'

In [ ]:
def process_func(example):
    MAX_LENGTH = 1024 # 设置最大序列长度为1024个token
    input_ids, attention_mask, labels = [], [], [] # 初始化返回值
    # 适配chat_template
    instruction = tokenizer(
        f"<s><|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n"   # 这里自定义系统指令，因为想要微调的是模型的语气风格，告诉模型扮演什么
        f"<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n"  
        f"<|im_start|>assistant\n<think>\n\n</think>\n\n",  
        add_special_tokens=False   
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    # 将instructio部分和response部分的input_ids拼接，并在末尾添加eos token作为标记结束的token
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]  # 这个是tokenizer.pad_token对应的id，即<|endoftext|>对应的id
    # 注意力掩码，表示模型需要关注的位置
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    # 对于instruction，使用-100表示这些位置不计算loss（即模型不需要预测这部分）
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]    
    # 这样label就是一个完整的instruction + response的拼接，且instruction部分的inputs id全为-100

    if len(input_ids) > MAX_LENGTH:  # 超出最大序列长度截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
# 预处理函数返回的仍然是索引，需要decode

In [8]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3729/3729 [00:02<00:00, 1316.63 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [ ]:
# 这是处理过后数据集的效果，符合模版
print(tokenizer.decode(tokenized_id[0]['input_ids']))

<s><|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛<|im_end|>
<|im_start|>user
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|im_end|>
<|im_start|>assistant
<think>

</think>

嘘——都说许愿说破是不灵的。<|endoftext|>


In [ ]:
# 这是label的效果，忽略掉前面值为-100的部分，只解码后面有效值部分，可以发现label就是原始数据集里的output
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"]))))

嘘——都说许愿说破是不灵的。<|endoftext|>


In [21]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",dtype=torch.bfloat16)
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [ ]:
#prompt = "小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——"
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template(
                                    [{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}], 
                                    add_generation_prompt=True,  # 加上<|im_start|>assistant，告诉模型从哪返回回复
                                    tokenize=True,
                                    return_tensors="pt",
                                    return_dict=True,
                                    enable_thinking=False
                                )
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<think>
嗯，用户问我是谁，作为甄嬛，我需要以皇帝身边的女人身份来回应。首先，我应该保持优雅和神秘，但又不失亲近感。可以提到自己的名字，比如“嬛”，然后表达对皇帝的敬意和对宫中的职责。同时，可以暗示自己有特殊的能力或地位，比如“嬛嬛，陛下之宠”。这样既符合角色设定，又能自然地引导对话继续。

接下来，我需要考虑如何展开对话。可以询问用户对我的了解，或者分享一些宫中的趣事，让对话更生动。同时，保持语言的简洁和优雅，避免过于复杂的句子。最后，确保回应符合甄嬛的性格特点，既有智慧又有魅力。
</think>

嬛嬛，陛下之宠。我名甄嬛，陛下之女。宫中之事，皆以我为知。若有所问，可随我言。


In [ ]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [17]:
# LoRA部分
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # 要在原始模型的哪些层插入LoRA adapter
    inference_mode=False, # 关闭推理模式
    r=8,                  # Lora 秩
    lora_alpha=32,        # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1      # Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'q_proj', 'gate_proj', 'v_proj', 'up_proj', 'down_proj', 'o_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [ ]:
model = get_peft_model(model, config) #  get_peft_model会自动根据config的内容在基础模型model里插入LoRA adapter
model  # 模型从原本的Qwen3类变成了PeftModelForCausalLM类， 可以通过 model.print_trainable_parameters() 查看可训练参数

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [23]:
model.print_trainable_parameters()

trainable params: 5,046,272 || all params: 601,096,192 || trainable%: 0.8395


In [ ]:
# 配置训练参数
args = TrainingArguments(
    output_dir="./output/Qwen3_8B_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [ ]:
import swanlab
from swanlab.integration.transformers import SwanLabCallback

swanlab.login(api_key="")

swanlab.init(
    project="Finetune-Qwen3-1.5B-with-cos",
    experiment_name = "Test1",
    logdir = "logs",
    config={
        "learning_rate": 1e-4,
        "architecture": "Qwen3-1.5B",
        "dataset": "huanhuan.json",
        "per_device_train_batch_size": 4,
        "gradient_accumulation_steps": 4,
    }
)

c:\ProgramData\miniconda3\envs\clip\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

swanlab: Tracking run with swanlab version 0.6.10

swanlab: Run data will be saved locally in 
e:\pythonCode\FineTune\Qwen\logs\run-20250924_165528-eox9x7lqpys5tbnw5a4j3

swanlab: 👋 Hi Kang_RX,welcome to swanlab!

swanlab: Syncing run Test1 to the cloud

swanlab: 🏠 View project at https://swanlab.cn/@Kang_RX/Finetune-Qwen3-1.5B-with-cos

swanlab: 🚀 View run at https://swanlab.cn/@Kang_RX/Finetune-Qwen3-1.5B-with-cos/runs/eox9x7lqpys5tbnw5a4j3

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[SwanLabCallback()]
)

In [ ]:
trainer.train()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = f'E:\pythonCode\FineTune\Qwen\Qwen2.5_1.5B'
lora_path = './output/Qwen3_8B_lora/checkpoint-699' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

In [ ]:
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template(
                                    [{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                    add_generation_prompt=True,  # 加上<|im_start|>assistant，告诉模型从哪开始
                                    tokenize=True,
                                    return_tensors="pt",
                                    return_dict=True,
                                    enable_thinking=False
                                )


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))